In [ ]:
#%matplotlib inline

#initial_state_vector = np.concatenate(rr_earth.value, vv_earth.value/24, axis=0)

# Mangalyaan 
# Launch_date = 5 November 2013, 09:08 UTC
#date_launch = ast_time.Time('2013-11-05 09:08', scale='utc')
#date_arrival = ast_time.Time('2014-09-24 01:47', scale='utc')
#print(date_launch)
#[X, fitness] = lamberts_problem_solver(date_launch, date_arrival)
#print(X, fitness)
#a = test1()


In [ ]:
%load_ext line_profiler
%matplotlib inline
import random
import subprocess
import multiprocessing as mp
from multiprocessing import Lock
import time
import random
import sys
import os
import errno
import shutil 
import subprocess
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import cma
from decimal import Decimal
from astropy import units as u
from poliastro.bodies import Earth, Sun
from poliastro.twobody import Orbit
from poliastro.plotting import plot
from poliastro.plotting import OrbitPlotter
from numpy import linalg as LA
from mpl_toolkits.mplot3d import Axes3D
import astropy.units as u
from astropy import time as ast_time
from poliastro import iod
from astropy.coordinates import solar_system_ephemeris
from poliastro.ephem import get_body_ephem
from astropy import time as ast_time
from lamberts_problem_solver import *
import lamberts_problem_solver as lps
import atexit
import transfer_orbit_plotter as top

    
def plot_transfer_orbit(launch, arrival, v):
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection='3d')
    
    # number to a data time type
    launch_date_jd = ast_time.Time(launch, format='jd', scale='utc')
    arrival_date_jd = ast_time.Time(arrival, format='jd', scale='utc')  
    # JD to ISO (UTC)
    launch_date_iso = launch_date_jd.iso
    arrival_date_iso = arrival_date_jd.iso
    top.transfer_orbit_plotter(launch_date_iso, arrival_date_iso, v*(u.kilometer/u.second))

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def set_master_integrator_file(launch_date,arrival_date):
    launch_date_jd = ast_time.Time(launch_date, format='jd', scale='utc')
    arrival_date_jd = ast_time.Time(arrival_date, format='jd', scale='utc')
    launch_date_iso = launch_date_jd.iso
    arrival_date_iso = arrival_date_jd.iso


def find_boundary_dates():
    
   # global X
    switch = 0
    deltav_pass = 4
    # Launch time and Arrival time are taken as unknown.
    # The limits for launch time is taken as the complete window of Earth-Mars minimum distance encounter. 
    # Usually around 2.3 years. Arrival date is taken > arrival date + 200 days atleast. So we will put 
    # this condition inside the while loop for selective children formations. 
    # Have to use Julian dates. To make the es work easily.
    # Launch date: [07-07-2017 00:00:00.0, 07-01-2020 00:00:00.0] in JD : [2457941.5, 2458855.5]
    # Arrival Date: [07-07-2017 00:00:00.0 + 200days, 07-01-2020 00:00:00.0 + 400days] in JD [2458141.5, 2459255.5]
    
    # Kai : 2461587, 2463598 
    # scale (2463598 - 2461587)
    
    filename = '../output/candidates.txt'
    try:
        os.remove(filename)
    except OSError:
        pass
    
    f = open(filename, "ab")    
    sol_cand = []
    
    time_min_l = 2461587   # 30 June 2027
    time_max_l = 2463600   # 02 Jan 2033 
    time_min_a = 2461787   # 16 January 2028
    time_max_a = 2464800   # 16 april 2036
    
    # Kai plot starting dates [2462486, 2462729]
    #Best Results: AdaptSigmaNone, cma_on:True, ['CMA_stds'] = [10,10,8,8,8] 
    #    opts['CMA_mu'] = 10, opts['popsize'] = 50
    opts = cma.CMAOptions()
    opts['seed'] = 693169
    opts['CMA_mu'] = 2
    opts['popsize'] = 15
    opts['bounds'] = [[time_min_l,time_min_a, -deltav_pass,-deltav_pass,-deltav_pass], [time_max_l,time_max_a,deltav_pass,deltav_pass,deltav_pass]]
    opts['CMA_stds'] = [20,20,(deltav_pass-1)*2,(deltav_pass-1)*2,(deltav_pass-1)*2]
    opts['AdaptSigma'] = 'CMAAdaptSigmaCSA'
    #opts['cs_multiplier'] = 1
    opts['CSA_dampfac'] = 4
    opts['ftarget'] = 0
    opts['maxiter'] = 1000
    opts['verb_append'] = 1
     
    #opts['minstd'] = 10
    #opts['maxstd'] = 30
    opts.set({'cma_on':True})
    #opts['cma_on'] = 'False'
    
    #opts['seed':1003916,'CMA_mu':2, 
    
     #   'bounds': [[time_min_l,time_min_a, -deltav_pass,-deltav_pass,-deltav_pass], [time_max_l,time_max_a,deltav_pass,deltav_pass,deltav_pass]],
      #                           'CMA_stds':[30,30,8,8,8], 'AdaptSigma':'CMAAdaptSigmaCSA',
       #                         'CSA_dampfac': 3, 'CSA_damp_mueff_exponent': 1,
        #                         'cma_on':True, 'CMA_active': 1,   
         #                       'ftarget': 1000, ]
    #es = cma.CMAEvolutionStrategy([g1 ,g2, 1, 1, 1], 10, {'seed':ran_seed, 'CMA_mu':2, 'popsize':10,
    #                            'bounds': [[-30,-30,-30,-30,-30], [30,30,30,30,30]],
    #                                               'CMA_stds':[100,100,20,20,20], 'ftarget': 0, 'maxiter':150,'verb_append':1})
  
    # Best Configuration for a line.:
        # cma_mu:5 , popsize:30,['CMA_stds'] = [10,10,6,6,6],['CSA_dampfac'] = 10, opts['maxiter'] = 300
        # no deltav_pass, fitness = term1 + 10**(term2 + 2)
    
    
    # ['AdaptSigma'] = 'CMAAdaptSigmaCSA'
    es = cma.CMAEvolutionStrategy([2462486, 2463029, 0, 0, 0], 1, opts)
    
    # 'AdaptSigma':cma.CMAAdaptSigmaNone,
    #logger = cma.CMADataLogger().register(es) 
    #opts.set('maxiter', 2)
    #opts['maxiter'] = 300 
    #es.opts.set({'maxiter': 50})
    #es.opts.set({'CMA_stds': [2,2,4,4,4]})
    #es.opts.set({'CSA_dampfac': 1})
    
    
    while not es.stop():
        X1, fit1 = [], []
        initial_velocity = []
        earth_position = []
        earth_velocity = []
        destination_vector = []
        dates = []
        sol_cand = []
        
        #es.AdaptSigma = 'cma.CMAAdaptSigmaTPA'
        #es.cma_on = True
        #print(es.cma_on)
        #if switch[0] == 1 and es.AdaptSigma == 'cma.CMAAdaptSigmaNone':
            #es.AdaptSigma = 'cma.CMAAdaptSigmaTPA'
            #print('Switch: {}'.format(switch[0]))
        len(X1)
        while len(X1) < es.popsize:
            X = es.ask(1,None,1)[0]
            #print(X)
            #ld = ((X[0] + 30)*(time_max_l-time_min_l))/60 + time_min_l
            #ad = ((X[1] + 30)*(time_max_a-time_min_a))/60 + time_min_a
            ld = X[0]
            ad = X[1]
            tof = ad - ld
            #print(tof)
            launch_date_jd = ast_time.Time(ld, format='jd', scale='utc')
            rr_earth, vv_earth = get_body_ephem("earth", launch_date_jd)
            
            v1 = [X[2], X[3], X[4]]
            v2 = vv_earth.to(u.km/u.second).value
            v1_u = unit_vector(v1)
            v2_u = unit_vector(v2)
            angle_dev =  np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
            
            #print(np.degrees(angle_dev))
            # Launch angle is fixed as if satellite is shooted within the limits of a cone 
            deltav = LA.norm(v1)
            #if (switch == 1):
                #print('{} {} {}'.format(np.degrees(angle_dev), tof, X))
            ##  
            if (np.degrees(angle_dev) < 45) and (1000 > tof > 200) and (deltav < deltav_pass):
                X1.append(X)
                initial_velocity.append(X[2:5])
                earth_position.append(rr_earth.to(u.km).value)
                earth_velocity.append(v2)
                
                arrival_date_jd = ast_time.Time(ad, format='jd', scale='utc')
                rr_dest, vv_dest = get_body_ephem("mars", arrival_date_jd)
                
                tmp_mars = np.concatenate((rr_dest.to(u.km).value,vv_dest.to(u.km/u.second).value), axis=0)
                destination_vector.append(tmp_mars)
                
                launch_date_iso = launch_date_jd.iso
                arrival_date_iso = arrival_date_jd.iso
                dates_tmp = [launch_date_iso, arrival_date_iso]
                dates.append(dates_tmp)
                
                sol_cand.append([ld,tof])
        
        np.savetxt(f ,sol_cand, fmt='%5.3f', delimiter=' ', newline='\n')
        
        final_vector = lps.process_splitter(initial_velocity, earth_position, earth_velocity, dates)
        #print('Real:{}  {}'.format(initial_velocity,final_vector))
        fit1 = lps.fitness_calculator_splitter(switch,initial_velocity, final_vector, destination_vector)
        
        #min_fit = np.amin(fit1)
        
        #check for acceptence
        #print(fit1)
        es.tell(X1, fit1)  
        #logger.add()
        es.disp(10)
    
    
    f.close()
    
    
    
    print('X: {0} and Fitness: {1}.\n'.format(X1,fit1)) 
    call_print_results(X1[0],fit1[0])
    #logger.plot()
    #cma.plot()
    
def call_print_results(X,fit):
    
    ld = X[0]
    ad = X[1]
    
    launch_date_jd = ast_time.Time(ld, format='jd', scale='utc')
    arrival_date_jd = ast_time.Time(ad, format='jd', scale='utc')
    launch_date_iso = launch_date_jd.iso
    arrival_date_iso = arrival_date_jd.iso
    
    print('Launch Date: {}'.format(launch_date_iso))
    print('Arrival Date: {}'.format(arrival_date_iso))
    print('TOF: {}'.format(arrival_date_jd - launch_date_jd))
    print('V: {} and Delta V : {}'.format(X[2:5], LA.norm(X[2:5])))    
    
    

def call_lamberts_problem_solver(launch_date, arrival_date, fitness_target = None):
    launch_date_jd = ast_time.Time(launch_date, format='jd', scale='utc')
    arrival_date_jd = ast_time.Time(arrival_date, format='jd', scale='utc')
    launch_date_iso = launch_date_jd.iso
    arrival_date_iso = arrival_date_jd.iso
    print('Launch Date: {}'.format(launch_date_iso))
    print('Arrival Date: {}'.format(arrival_date_iso))
    print('TOF: {}'.format(arrival_date_jd - launch_date_jd))
    
    if fitness_target is None:
        [va, fitness] = lamberts_problem_solver(launch_date_iso, arrival_date_iso)
    else:
        [va, fitness] = lamberts_problem_solver(launch_date_iso, arrival_date_iso, fitness_target)
    #print(X, fitness)
    
    return ([va[0], va[1], va[2], fitness])
    
def porkchop_list():
    
    deltav_max = 40
    catching_radius = 100           # in kms
    launch_date_min = 2457950.5     # 16July2017
    launch_date_max = 2458680.5     # 16july2019
    arrival_date_min = 2458150.5    # 16July2017 + 200 days
    arrival_date_max = 2459199.5    # 16December2020
    
    launch_date = launch_date_min
    
    lst = []
    
    while launch_date < launch_date_max:
        arrival_date = launch_date + 200
        
        while arrival_date < arrival_date_max:
            [vx, vy, vz, fitness] = call_lamberts_problem_solver(launch_date, arrival_date, catching_radius)
            if (LA.norm([vx, vy, vz]) < deltav_max):
                lst.append([launch_date, arrival_date, vx, vy, vz, LA.norm([vx, vy, vz]), fitness])
                print(lst)
            arrival_date = arrival_date+5
    
    print(lst)
    '''
    # find the best window
    transfer_orbit_plotter(date_launch, date_arrival, va*(u.kilometer/u.second))

    # porkchop_plotter
    '''
    
    # Launch time and Arrival time are taken as unknown.
    # The limits for launch time is taken as the complete window of Earth-Mars minimum distance encounter. 
    # Usually around 2.3 years. Arrival date is taken > arrival date + 200 days atleast. So we will put 
    # this condition inside the while loop for selective children formations. 
    # Have to use Julian dates. To make the es work easily.
    # Launch date: [07-07-2017 00:00:00.0, 07-01-2020 00:00:00.0] in JD : [2457941.5, 2458855.5]
    # Arrival Date: [07-07-2017 00:00:00.0 + 200days, 07-01-2020 00:00:00.0 + 400days] in JD [2458141.5, 2459255.5]

def operation_selector(ch):
    if ch == 1:
        catching_radius = 100           # in kms
        # MSL launch: 26 Nov 2011 15:02:0.211        #Mangalyaan launch:
        launch_date = 2462759.5 
        # MSL arrival: 06 Aug 2012 05:17             #Mangalyaan arrival:
        arrival_date = 2463659.5
        [vx, vy, vz, fitness] = call_lamberts_problem_solver(launch_date,arrival_date,catching_radius)
        print('Velocity Vector[km/s] w.r.t. Earth = {}, {}, {}'.format(vx,vy,vz))
        
        launch_date_jd = ast_time.Time(launch_date, format='jd', scale='utc')
        arrival_date_jd = ast_time.Time(arrival_date, format='jd', scale='utc')
        rr_earth, vv_earth = get_body_ephem("earth", launch_date_jd)
        v_earth = vv_earth.to(u.kilometer/u.second).value
        print('Velocity of satellite w.r.t Sun = {}, {}, {}.'.format(vx+v_earth[0], vy+v_earth[1], vz+v_earth[2]))
        
        va = [vx, vy, vz]
        #plot_transfer_orbit(launch_date, arrival_date, va)
            
    elif ch == 2:
        porkchop_list()
    elif ch == 3:
        find_boundary_dates()
    else:
        print('invalid choice value in choice')


if __name__ == '__main__':
    
    # 1 : lamberts problem solver and 2 for porkchop_list
    
    choice = 3
    operation_selector(choice)
    #%prun find_boundary_dates()
    path = '../workspace'
    remove_temp_dir(path)
    print('Completed: Workspace cleaned!')        

    '''    
    try:
        operation_selector(choice)
        path = '../workspace'
        remove_temp_dir(path)
        print('Completed: Workspace cleaned!')        
    except:
        path = '../workspace'
        remove_temp_dir(path)
        print('Interrupt/error: Workspace cleaned!')
    '''

mu = 2.000000
(2_w,15)-aCMA-ES (mu_w=1.9,w_1=60%) in dimension 5 (seed=693169, Thu Aug 24 19:49:29 2017)


WARNING (module=cma):  function values are not a list of scalars (further warnings are suppressed)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     16 1.404475431700395e+08 1.0e+00 9.47e-01  5e+00  2e+01 0:0.1
WARNING (module=cma):  function values are not a list of scalars (further warnings are suppressed)
    2     31 1.465130446613657e+08 1.1e+00 9.26e-01  5e+00  2e+01 0:1.4
    3     46 1.398217208831168e+08 1.1e+00 9.02e-01  5e+00  2e+01 0:2.6
   10    151 1.092092534444740e+08 1.4e+00 7.69e-01  4e+00  1e+01 0:11.5
   20    301 7.550788342744689e+07 2.0e+00 6.64e-01  2e+00  1e+01 0:24.0
   30    451 4.614464417200935e+07 2.7e+00 5.87e-01  2e+00  9e+00 0:36.0
   40    601 4.216016840247158e+07 6.9e+00 4.66e-01  7e-01  8e+00 0:47.3
   50    751 3.803951933128974e+07 1.8e+01 4.54e-01  5e-01  1e+01 0:58.1
   60    901 2.825983801719008e+07 2.6e+01 4.74e-01  5e-01  8e+00 1:8.5
   70   1051 2.551640549943801e+07 4.8e+01 4.79e-01  3e-01  1e+01 1:18.6
   8

In [ ]:
#x = [np.array([1,2]),np.array([2,3]),np.array([3,4])]
#print(x[1][0])
#-17.51555934,  16.3412824
#Closest Results: 24 Sept 2030 TOF: 854   v(w.r.t) : -0.54421463,  3.84872828, -0.56233629
from astropy import time as ast_time

ld = 2462846
ad = 2463846
print(ld)
print(ad)

launch_date_jd = ast_time.Time(ld, format='jd', scale='utc')
arrival_date_jd = ast_time.Time(ad, format='jd', scale='utc')
launch_date_iso = launch_date_jd.iso
arrival_date_iso = arrival_date_jd.iso
print('Launch Date: {}'.format(launch_date_iso))
print('Arrival Date: {}'.format(arrival_date_iso))
print('TOF: {}'.format(arrival_date_jd - launch_date_jd))